# 4. 順運動学、逆運動学を用いた動作生成

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>順運動学、逆運動学を使ったHSRのアームの駆動方法を学習します</p>
</div> 

# 順運動学を用いた動作生成

「順運動学」とは、関節の変位からロボットの手先の位置・姿勢を求めることです。アームの各関節角度を指定してHSRを動かしてみましょう。

必要なライブラリのインポートと、初期化を行います。

In [ ]:
import math
import moveit_commander
import rospy
import tf
from utils import *
rospy.init_node("arm")

アームとして定義されている各関節の名称を配列として取得します。以降の各コマンドでは、この配列の順序に対応させて各関節値を設定します。

ここで`arm`はロボットのアームの関節角を管理する変数です。

In [ ]:
arm.get_active_joints()

各関節に対応するリンクがロボットのどの位置にあるか確認しましょう。

rvizを立ち上げて、「TF」を展開して「Frames」から興味のあるリンク名にチェックを入れてその位置を確認しましょう。

In [ ]:
%%script bash --bg
rviz -d data/4_arm.rviz > /dev/null 2>&1

![title](./imgs/4_model_and_tf.png)

各座標は赤緑青(RGB)の順番でx軸、y軸、z軸を表しています。

各関節角の現在値を取得します。関節角度の単位はラジアンで返ってきます。

In [ ]:
arm.get_current_joint_values()

試しに指定した関節値にアームを駆動してみます。

まずは、目標の関節値を設定します。

In [ ]:
arm.set_joint_value_target([0, 0, 0, 0, 0, 0])

目標を設定後「go()」コマンドを実行することで、アームが動きます。

In [ ]:
arm.go()

例えば、以下のように設定すると昇降軸を動作させることができます。

In [ ]:
arm.set_joint_value_target([0.5, 0, 0, 0, 0, 0])
arm.go()

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>「モノを拾う」アームの動きを作ってみましょう。</p>
</div> 

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


# 逆運動学を用いた動作生成

アームの各関節を設定して動きを作るのは案外大変です。「逆運動学」を使うと手先の目標座標から各関節に設定すべき角度を自動で算出することができます。

「順運動学を用いた動作生成」では、HSRのアームのみを動かしていました。しかし、アームだけでは遠くの物を掴めないなどの問題があります。

HSRはモバイルマニピュレータと呼ばれ、台車の上にアームが搭載されています。以降は、台車とアームを用いた全身の制御を行います。

ロボットの手先(エンドエフェクタ)の名前を確認します。アームの先にあるハンドのリンクが設定されているはずです+。

ここで`whole_body`はロボットの頭を除く全関節角を管理する変数です。

In [ ]:
whole_body.get_end_effector_link()

## 各関節情報の確認

ロボットの各関節の角度は、現在、以下の値になっています。

In [ ]:
whole_body.get_current_joint_values()

上記の関節角度は、以下の各関節名に対応します。

In [ ]:
whole_body.get_joints()

ここで、ロボットの構造は既知なので、ロボットの構造に現在の各関節角度を当てはめ、座標変換を繰り返せばロボットの手先座標を計算できます。

HSRの構造情報を表示させてみましょう。

リンクの親子関係と、その相対座標を表示した画像が得られます。四角形で表されているのがリンクで、円で表されているのが関節です。

大きな画像なので、スクロールして閲覧してください。

In [ ]:
!rosparam get -p /robot_description > /tmp/robot.urdf
!urdf_to_graphiz /tmp/robot.urdf && dot -T png hsrb.gv -o /tmp/robot.png
from IPython.display import Image
with open('/tmp/robot.png','rb') as file:
    display(Image(data=file.read(), unconfined=True))

以下のコマンドを実行することで、順運動学によって計算されたエンドエフェクタ('hand_palm_link')の位置姿勢を知ることができます。

In [ ]:
get_relative_coordinate("map", "hand_palm_link")

手先座標は、3次元の位置と四元数（quaternion）による姿勢で表現されています。四元数の詳細が気になる方は調べてみてください。

## 逆運動学の計算

目標とする手先座標を与えられた時に、そこからアームの各関節角を逆算することを「逆運動学を解く」と言います。

まずは目標の手先座標を設定します。「map」という座標を基準にして、位置：(x,y,z)=(0.2,0.4,1.0)、姿勢：(roll, pitch, yaw)=(180, 90, 0)をエンドエフェクタの目標値としています。

ここで姿勢はx軸, y軸, z軸の順番でroll, pitch, yawの値で回転させ、その値を四元数に変換しています。

In [ ]:
from geometry_msgs.msg import PoseStamped

p = PoseStamped()

# 基準座標を設定
p.header.frame_id = "/map"

# 位置を設定
p.pose.position.x = 0.2
p.pose.position.y = 0.4
p.pose.position.z = 1.0

# ロール、ピッチ、ヨーの順番で回転し、クオータニオンに変換
p.pose.orientation = quaternion_from_euler(180, 90, 0)

# 目標位置姿勢をセット
whole_body.set_pose_target(p)

目標を設定後、以下を実行すると、逆運動学の計算が始まります。

計算が成功すると算出された軌道が返ってきます。`plan`には、各時間ごとの全関節角の目標値の情報(時間軌道と呼びます)が詰まっています。

逆運動学の解が得られなかった場合（目標の手先座標の設定が適切でなかった場合）は、長さゼロの配列が返ってきますので、目標を修正して再計算してください。

In [ ]:
plan = whole_body.plan()
plan

算出された時間軌道を実際に実行します。

生成された軌道を引数として与えて以下を実行することで、アームが逆運動学の計算結果に従って動きます。

In [ ]:
whole_body.execute(plan)

「TF」を展開して、「Frames」の中の「map」と「hand_palm_link」を表示させてみましょう。

エンドエフェクタが目標の位置姿勢になっていることを確認しましょう。

![title](./imgs/4_map_ee_tf.png)

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>エンドエフェクタの目標位置姿勢を変えて、ロボットを動かしてみましょう。</p>
</div>

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


# レゴブロックを拾ってみる

上記の処理をまとめたmove_wholebody_ikという関数を用いてみましょう。引数は3次元位置と手先の回転角度です。

```python
import moveit_commander
from geometry_msgs.msg import PoseStamped

# moveitでの制御対象として全身制御を指定
whole_body = moveit_commander.MoveGroupCommander("whole_body_light")
whole_body.allow_replanning(True)
whole_body.set_workspace([-3.0, -3.0, 3.0, 3.0])

def move_wholebody_ik(x, y, z, roll, pitch, yaw):
    u"""ロボットを全身の逆運動学で制御する関数

    引数：
        x (float): エンドエフェクタの目標x値 [m]
        y (float): エンドエフェクタの目標y値 [m]
        z (float): エンドエフェクタの目標z値 [m]
        roll (float): エンドエフェクタの目標roll値 [deg]
        pitch (float): エンドエフェクタの目標pitch値 [deg]
        yaw (float): エンドエフェクタの目標yaw値 [deg]

    返り値:
        正しく動作すればTrue, そうでなければFalse

    """

    p = PoseStamped()

    # "map"座標を基準座標に指定
    p.header.frame_id = "/map"

    # エンドエフェクタの目標位置姿勢のx,y,z座標をセットします
    p.pose.position.x = x
    p.pose.position.y = y
    p.pose.position.z = z

    # おリラー角をクオータニオンに変換します
    p.pose.orientation = quaternion_from_euler(roll, pitch, yaw)

    # 目標位置姿勢をセット
    whole_body.set_pose_target(p)
    return whole_body.go()

```

関数内でオイラー角から四元数に変換することで、人間が理解しやすいオイラー角によって手先の方向を設定できるようにしています。

例えば、上から物を掴みたい時は(roll, pitch, yaw) = (180,0,手先の回転角度)、横からは(180,90,手先の回転角度)というように設定すれば良いです。

シミュレータ内に物体を出現させて、HSRに拾わせてみましょう。以下のコマンドでどのような物体があるか表示します。

In [ ]:
get_object_list()

今回は、レゴブロック「e_lego_duplo」をHSRの前方0.4mの位置に出現させます。

In [ ]:
# レゴブロックを(x, y, z) = (0.4, 0.0, 0.0)の位置に出現させる
put_object("e_lego_duplo", 0.4, 0.0, 0.0)

ハンドを最大角度まで開きます。

In [ ]:
# 引数として1.0を与えると開き、0.0を与えると閉じる
move_hand(1.0)

逆運動学を使ってHSRの手先を前方0.4mの位置に動かします。

In [ ]:
move_wholebody_ik(0.45, -0.05, 0.1, 180, 0, 0)

ハンドを閉じます。

In [ ]:
move_hand(0.0)

アームを初期姿勢に戻します。

In [ ]:
move_arm_init()

レゴブロックを取るのに失敗した場合、以下のコマンドでレゴブロックを消して、再度出現させると元の取りやすい位置に戻ります。再挑戦してみましょう。

In [ ]:
delete_object("e_lego_duplo")

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>カンを机の上に場所に出現させて、同様にカンを拾うプログラムを書いてみましょう。</p>
    <p>下のコマンドでカンを出現させたり、消したりすることができます。</p>
</div>

In [ ]:
put_object("tomato_soup_can", 1.1, 1.65, 0.45)

In [ ]:
delete_objecta("tomato_soup_can")

In [ ]:
# 自分で考えてみましょう。この下に入力できます。


<div class="alert alert-block alert-info">
    <b>次の学習</b>
    <p>この章では物体の位置を人間が教えてあげていました。</p>
    <p>次章ではカメラ画像から物体を認識し、自動で物体を拾うプログラムを書いてみましょう。</p>
</div>